In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import requests
import io
import datetime

# --- Параметры для запроса к API Яндекс.Метрики ---
id_cab = х 
API_URL = '.csv'
API_TOKEN = 'х'
date_from = "2025-01-01"
date_to = datetime.date.today().strftime("%Y-%m-%d")  # Текущая дата в формате YYYY-MM-DD

# --- Цели для воронки ---
goal_ids = {  # Словарь с ID целей и их названиями
    'open_dialog_register': х, #на входе
    'open_form_submit_confirm_mail': х, #подтвердили почту
    'form_submit_register': х, #успешная регистрация
    'open_dialog_login': х, # Войти в личный кабинет
    'form_submit_login': х, # успешная авторизация
    'symptomov.net_vitrina': х, # Посещение страницы Витрина
    'click_link_instruction_for_use__vitrina': х, # Клик на ссылку 
    'click_drug_properties__vitrina': х, # Клик на кнопку 
    'click_drug_properties__vitrina_instruktsiya': х, # Клик на кнопку "
    'click_order_online__vitrina' : х, #Клик на кнопку 
    'market_widget_init': х, #Инициализация виджета
    'market_widget_load': х, #Загрузка виджета
    'market_widget_show': х, #Показ виджета
    'market_widget_click': х, # Клик по виджету 
    'click_zdravcity__vitrina_gde-kupit': х, #Смотрите 
    'click_uteka__vitrina_gde-kupit': х, #Смотрите  
    'click_ozon__vitrina_gde-kupit': х, #Смотрите   
    'click_apteka_ru__vitrina_gde-kupit': х, #Смотрите  

}

# --- Метрики и измерения ---
metrics = ["ym:s:users"]  # Общее количество пользователей
for goal_name, goal_id in goal_ids.items():
    metrics.append(f"ym:s:goal{goal_id}users")  # Метрики для каждой цели

dimensions = [
    "ym:s:datePeriodMonth",  # Детализация по месяцам
    "ym:s:deviceCategory",  # Группировка по устройствам
]

params_funnel = {
    'date1': date_from,
    'date2': date_to,
    'id': id_cab,
    'accuracy': 1,
    'attribution': 'lastsign',
    'metrics': ','.join(metrics),
    'dimensions': ','.join(dimensions),
    'group': 'month',
    'filters': "ym:s:isRobot=='No'"
}
API_URL = 'data'  # URL для запроса

# --- Запрос к API ---
try:
    r = requests.get(API_URL, params=params_funnel, headers={'Authorization': 'OAuth ' + API_TOKEN})
    r.raise_for_status()  # Проверка на ошибки HTTP

    # --- Обработка данных ---
    json_data = r.json()
    data = json_data.get('data', [])

    if data:
        metric_columns = ["Users"] + list(goal_ids.keys())

        # Обработка метрик: приводим все списки к одинаковой длине
        metrics_list = [item.get('metrics', []) for item in data]
        max_len = max(len(x) for x in metrics_list)
        metrics_list_fixed = [x + [0] * (max_len - len(x)) for x in metrics_list]
        df_metrics = pd.DataFrame(metrics_list_fixed, columns=metric_columns[:max_len])

        # Обработка измерений (dimensions)
        dimensions_list = []
        for item in data:
            dims = item.get('dimensions', [])
            if dims:
                dims_names = [dim.get('name', None) for dim in dims]
            else:
                dims_names = [None, None]
            dimensions_list.append(dims_names)

        df_dimensions = pd.DataFrame(dimensions_list, columns=['Date interval of visit', 'Device type'])

        # Объединяем метрики и измерения
        df = pd.concat([df_metrics, df_dimensions], axis=1)

        # Заполняем пропущенные значения нулями
        df.fillna(0, inplace=True)

        # Преобразуем все числовые столбцы (кроме измерений) в целые числа
        num_cols = df_metrics.columns.tolist()
        df[num_cols] = df[num_cols].astype(int)

        # Функция для преобразования интервала в дату первого дня месяца в формате «01.03.2025»
        def convert_to_first_day(date_interval):
            if isinstance(date_interval, str):
                # Разделяем по " – " или " - " (учитываем разные типы дефисов)
                if ' – ' in date_interval:
                    start_date_str = date_interval.split(' – ')[0]
                elif ' - ' in date_interval:
                    start_date_str = date_interval.split(' - ')[0]
                else:
                    start_date_str = date_interval  # Если нет разделителя, берем всё как дату
                date = pd.to_datetime(start_date_str)
                return date.strftime('%d.%m.%Y')
            return date_interval

        # Применяем функцию к столбцу
        df['Date interval of visit'] = df['Date interval of visit'].apply(convert_to_first_day)

    else:
        df = pd.DataFrame()

except requests.exceptions.RequestException as e:
    print("Ошибка запроса:", e)
    df = pd.DataFrame()

# Вывод итогового DataFrame
print(df)


    Users  open_dialog_register  open_form_submit_confirm_mail  \
0    1674                     0                              0   
1     887                     0                              0   
2     639                     1                              0   
3     637                     0                              0   
4     456                     0                              0   
5     158                    13                              1   
6     129                     0                              0   
7     119                    15                              8   
8      64                     0                              0   
9      58                     0                              0   
10     46                     0                              0   
11     42                     5                              0   
12     30                     0                              0   
13     23                     1                              0   
14     13 